In [56]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [57]:
train_set_path = "/Users/weizhuozhang/Desktop/Files/train_encode_tuned.npy"
dev_set_path = "/Users/weizhuozhang/Desktop/Files/dev_encode_tuned.npy"
train_set_csv = "/Users/weizhuozhang/Desktop/Files/train_set_v1.txt"
dev_set_csv = "/Users/weizhuozhang/Desktop/Files/dev_set_v1.txt"

train_set = np.load(train_set_path)
train_csv = pd.read_csv(train_set_csv, sep='\t', header=None)
train_csv = np.array(train_csv)
dev_set = np.load(dev_set_path)
dev_csv = pd.read_csv(dev_set_csv, sep='\t', header=None)
dev_csv = np.array(dev_csv)

train_label = (train_csv[:,0]).astype('int')
dev_label = (dev_csv[:,0]).astype('int')

In [58]:
import time
k_neighbors = [3, 7, 9, 13, 15]
for k in k_neighbors:
    knn_clf = KNeighborsClassifier(n_jobs=-1, n_neighbors=k)
    knn_clf.fit(train_set, train_label)
    time_start=time.time()
    predicted = knn_clf.predict(dev_set)
    accuracy = sum(predicted == dev_label)/len(predicted)
    time_end=time.time()
    print("---------------------------------")
    print("knn k={0}".format(k))
    print("accuracy: {0}".format(accuracy))
    print("Time spent: {0:.2f}ms".format((time_end-time_start)*1000))


CPU times: user 37.2 s, sys: 2.74 s, total: 39.9 s
Wall time: 41.7 s


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [59]:
%%time
predicted = knn_clf.predict(dev_set)
#accuracy = sum(predicted == dev_label)/len(predicted)

KeyboardInterrupt: 

In [55]:
#print(accuracy)

0.1691949950171631
